In [ ]:
import pandas as pd
import os
os.chdir('/content/drive/MyDrive/environment/전처리 데이터 ')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import unicodedata

a_normalized = unicodedata.normalize('NFC', os.listdir()[1])  # https://byeolcardi.tistory.com/24
print(a_normalized[0])
print(os.listdir()[1][0])

자
ᄌ


In [ ]:
os.listdir()

['df 통합.ipynb',
 '송파구.csv',
 '은평구.csv',
 '중구.csv',
 '서대문구.csv',
 '성동구.csv',
 '영등포구.csv',
 '강북,관악,구로,동대문,마포,서초구 흡연시설.csv',
 '광진구.csv',
 '중랑구.csv',
 '강서구.csv',
 '성북구.csv',
 '양천구.csv',
 '용산구.csv',
 '지오코딩 전.csv',
 '지오코딩 전.gsheet',
 '지오코딩 전__.csv']

In [ ]:
### 칼럼변경 필요한 data 처리

칼럼변경 = ['영등포구', '성동구','중구','서대문구','송파구','은평구']
칼럼변경 = list(map(lambda x: x+'.csv', 칼럼변경))
a = [pd.read_csv(i) for i in 칼럼변경]
a = pd.concat(a)
a.rename(columns = {'도로명주소':'주소'}, inplace=True)

###
b = pd.read_csv('강북,관악,구로,동대문,마포,서초구 흡연시설.csv')
b.rename(columns={'시설형태(개방/폐쇄형)':'시설형태'}, inplace=True)

In [ ]:
import glob
csv_list = [unicodedata.normalize('NFC', i) for i in glob.glob('*.csv')]
csv_list = [i for i in csv_list if (i not in 칼럼변경) and i != '강북,관악,구로,동대문,마포,서초구 흡연시설.csv']

df_list = []
for i in csv_list:
  try:
        df_list.append(pd.read_csv(i))
  except:
    try:
      df_list.append(pd.read_csv(i, encoding='cp949'))
    except Exception as e:
      print(i, e)

In [ ]:
c = pd.concat(df_list).iloc[:,1:]
df = pd.concat([a,b, c])
df['자치구'] = df['자치구'].apply(lambda x: x.split()[0] if len(x.split()) == 1 else x.split()[1])

In [ ]:
df = pd.read_csv('지오코딩_후_오류포함.csv', encoding='cp949')

In [ ]:
# 데이터 정제
# 14
df.loc[14, '위도'] = df.loc[14,'Latitude']
df.loc[14, '경도'] = df.loc[14,'Longitude']
df.loc[14, '주소'] = '서울특별시 영등포구 의사당대로 143 (여의도동)'

# 461
df.loc[461,'주소'] = df.loc[461,'설치위치']
df.loc[461, '설치위치'] = '호텔'

# 1011
df.loc[1011, '주소'] = df.loc[1011, '주소'].replace('?', ' ')
df.loc[1017, '주소'] = df.loc[1017, '주소'].replace('울','서울').replace('?',' ')

# 1066
df.loc[1006, '주소'] = df.loc[1006, '주소'].replace('?', ' ')

# 건국대학교 도로명주소 기입 - 확실하지 않음 (건국대학교 주소에 '관'으로 추정되는 숫자가 적혀있어서 건국대 도로명 주소 + '관' 추가함)
a = df.loc[df['설치위치'] == '건국대학교', '주소']
a = a.apply(lambda x: x.split()[0]+'관 '+ ' '.join(x.split()[1:]) if x[0].isdigit() else x)
df.loc[df['설치위치'] == '건국대학교', '주소'] = a.apply(lambda x: '서울특별시 광진구 능동로 120 '+x)

In [ ]:
df[df[['Latitude', 'Longitude']].isna().any(axis=1)].to_csv('지오코딩_전_오류만.csv', index=False)

In [ ]:
df = pd.read_csv('지오코딩_후_오류포함.csv', encoding='cp949')
df_오류만 = pd.read_csv('지오코딩_후_오류만.csv')

In [ ]:
df.iloc[df[['Latitude', 'Longitude']].isna().any(axis=1)] = df_오류만
df.drop(['위도', '경도'], axis=1, inplace=True)

df.rename(columns={"Latitude":'위도', 'Longitude':'경도'}, inplace=True)
df.to_csv('**서울시_흡연구역.csv', index=False)

위경도 잘못된 데이터 확인

In [ ]:
df= pd.read_csv('최종_서울시_흡연구역.csv')

def encode(df, data):
  자치구 = df.loc[df['주소'] == data, '자치구'].values[0]
  if '로' in data:
    if 자치구 in data:
      return '서울특별시 ' + data
    else:
      return '서울특별시 ' + 자치구 + ' ' + data
  elif '구' in data:
      return '서울특별시 ' + data
  else:
    return data

도로명주소_잘못 = df[~df['주소'].str.startswith('서울')]
도로명주소_잘못['주소'] = 도로명주소_잘못['주소'].apply(lambda x: encode(도로명주소_잘못, x))

In [ ]:
정제data = pd.read_csv('fin_지오.csv')
정제data['index'] = 도로명주소_잘못.index
정제data.set_index('index', inplace=True)

In [ ]:
df.loc[정제data.index, '주소'] = 정제data['주소']
df.loc[정제data.index, '위도'] = 정제data['Latitude']
df.loc[정제data.index, '경도'] = 정제data['Longitude']

In [ ]:
df[['위도','경도']].round(2).value_counts().sort_index()

위도     경도    
37.47  126.94     1
       126.98     1
       127.03     2
       127.04     8
       127.05     4
37.48  126.89     4
       126.90     5
       126.93     3
       126.95     4
       126.96     1
       126.98     5
       127.00     1
       127.03     3
       127.04    13
       127.05     1
       127.12     3
37.49  126.88     1
       126.89     2
       126.90     4
       126.93     1
       126.96     1
       127.01     1
       127.02     1
       127.03     1
37.50  126.87     2
       126.90     1
       127.00     3
       127.01     1
       127.02     1
       127.03     2
37.51  126.83     1
       126.84     1
       126.85     1
       126.89     2
       126.90     3
       126.91     1
       126.92     1
       127.01     1
       127.10     1
37.52  126.83     2
       126.84     1
       126.85     1
       126.86     2
       126.87     3
       126.88     3
       126.89     5
       126.90     6
       126.91     2
       126.92     6
      

In [ ]:
df.to_csv('최종_서울시_흡연구역.csv', index=False)

In [ ]:
# geopy보다 구글 api 가 더 잘됨/ 오류가 너무 자주 남

주소_nan = df[df[['Latitude', 'Longitude']].isna().any(axis=1)]
주소_nan_idx = 주소_nan.index

from geopy.geocoders import Nominatim
from tqdm import tqdm
import numpy as np

def geocode(address):
    try:
        geo_local = Nominatim(user_agent='South Korea')
        location = geo_local.geocode(address)
        geo = [location.latitude, location.longitude]
        return geo

    except:
        return [0,0]

for idx, addr in tqdm(zip(주소_nan_idx, df['주소'])):
    df.loc[idx,'latitude'] = geocode(addr)[0]
    df.loc[idx,'longitude'] = geocode(addr)[1]

16it [00:19,  1.20s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC+%EC%97%AC%EC%9D%98%EB%8F%84%EB%8F%99+%EC%97%AC%EC%9D%98%EB%8C%80%EB%A1%9C+128&format=json&limit=1
17it [00:21,  1.56s/it]WARNING:urllib3.connectionpool:Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C+%EC%98%81%EB%93%B1%ED%8F%AC%EA%B5%AC+%EC%97%AC%EC%9D%98%EB%8F%84%EB%8F%99+%EA%B5%AD%ED%9A%8C%EB%8C%80%EB%A1%9C76%EA%B8%B8+18&format=json&limit=1
41it [00:50,  1.23s/it]
